# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

from onnxruntime.transformers.models.stable_diffusion.diffusion_models import BaseModel

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import chromadb
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ConfigDict
from tavily import TavilyClient

from project.lib.agents import Agent
from project.lib.llm import LLM
from project.lib.messages import UserMessage, SystemMessage, BaseMessage, ToolMessage, AIMessage
from project.lib.tooling import tool, Tool
from project.lib.evaluation import EvaluationReport
from project.lib.memory import LongTermMemory, VectorStoreManager, MemoryFragment

In [3]:
load_dotenv()

True

In [4]:
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

In [5]:
class Game(BaseModel):
    """Game information"""
    model_config = ConfigDict(
        frozen=True,
    )

    Platform: str = Field(
        ...,
        description="Platform name of the game",
        alias="Platform",
        examples=["Game Boy", "Playstation 5", "Xbox 360"]
    )
    Name: str = Field(
        ...,
        description="Name of the Game",
        alias="Name",
        examples=["Super Mario World", "Pokémon Red and Blue", "Gran Turismo"]
    )
    YearOfRelease: int = Field(
        ...,
        description="Year when that game was released for that platform",
        alias="YearOfRelease",
        examples=[1990, 2001, 2010]
    )
    Description: str = Field(
        ...,
        description="Additional details about the game",
        alias="Description",
        examples=["A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser."]
    )

In [8]:
@tool
def retrieve_game(query: str, num_of_results = 5) -> list[Game]:
    """
    Semantic search: Finds most results in the vector DB

    args:
    - query: a question about game industry.
    - num_of_results: number of results to return

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    response = collection.query(query_texts=[query], n_results=num_of_results)

    if not (metadata := response.get("metadatas", [[]])[0]):
        return metadata

    return [Game(**meta) for meta in metadata]

In [9]:
@tool
def evaluate_retrieval(question: str, retrieved_docs: list[Game]) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.

    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    if not retrieved_docs:
        return EvaluationReport(
            useful=False,
            description="No documents were retrieved, so it's not possible to answer the question."
        )

    llm_evaluator = LLM()
    messages: list[BaseMessage] = [
        SystemMessage(
            content=(
                "You are an expert in evaluating documents."
                "Your task is to evaluate if the information in the documents are enough to respond the user question."
                "Give a detailed explanation, so it's possible to take an action to accept it or not"
            )
        ),
        UserMessage(
            content=(
                f"""
                    # Question: {question}
                    # Documents: {[doc.model_dump() for doc in retrieved_docs]}
                    """
            )
        )
    ]

    response = llm_evaluator.invoke(messages, response_format=EvaluationReport)

    if response.content:
        return EvaluationReport.model_validate_json(response.content)

    return EvaluationReport(
        useful=False,
        description="We were not able to evaluate the documents. Please try again."
    )

In [10]:
class SearchResult(BaseModel):
    """Search result"""
    model_config = ConfigDict(
        frozen=True,
    )

    title: str = Field(
        ...,
        description="Title of the search result",
        alias="title",
        examples=["Fifa 98"]
    )
    content: str = Field(
        ...,
        description="Content of the search result",
        alias="content",
        examples=["Fifa 98 was released in 1998."]
    )
    score: float = Field(
        default=0.0,
        description="Score of the search result",
        alias="score",
        examples=[0.9]
    )

In [42]:
@tool
def game_web_search(question: str) -> list[SearchResult]:
    """
    Semantic search: Finds most results in the vector DB

    args:
    - question: a question about game industry.
    """
    client = TavilyClient()
    response = client.search(question, num_results=5)

    return [SearchResult(**r) for r in response["results"]]

In [43]:
udaplay = Agent(
    model_name="gpt-5-nano-2025-08-07",
    instructions="""
    You are UdaPlay, an AI Research Agent for the video game industry.

    You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions.
    """,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    temperature=1,
)


In [44]:
user_queries = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X realeased for Playstation 5?",
]
session_id = "dcc120d2-3bcc-4543-afcb-207858823ab3"

for query in user_queries:
    run = udaplay.invoke(query, session_id=session_id)
    state = run.get_final_state()

    print("\n")
    print("-" * 80)
    print(f"Query: {query}")
    print(f"Response: {state['messages'][-1].content}")
    print("-" * 80)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


--------------------------------------------------------------------------------
Query: When Pokémon Gold and Silver was released?
Response: Pokémon Gold and Silver were released as follows:
- Japan: November 21, 1999
- North America: October 15, 2000
- Europe: November 25, 2000

If you want, I can add details about the games’ features or later re-releases.
--------------------------------------------------------------------------------
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


--------------------------------------------------------------------------------
Query: Which one was the first 3D platformer Mario game?
Response: Super Mario 64. It’s the first 3D platformer featuring Mario, released 

### (Optional) Advanced

In [45]:
# TODO: Update your agent with long-term memory
db = VectorStoreManager(os.getenv("OPENAI_API_KEY"))
_ = db.get_or_create_store("user_preferences")

long_term_memory = LongTermMemory(db)

# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes

In [46]:
def build_memory_registration_tool(ltm: LongTermMemory, owner: str, namespace: str):
    """
    Create a tool for agents to register new memories.

    This factory function creates a tool that allows AI agents to store new
    information about users in the long-term memory system. The tool is
    pre-configured with specific owner and namespace parameters.

    Args:
        ltm (LongTermMemory): The memory system instance to use
        owner (str): User identifier for memory ownership
        namespace (str): Namespace for organizing memories

    Returns:
        Tool: A configured tool for memory registration
    """
    def _register(content:str):
        ltm.register(
            MemoryFragment(
                content=content,
                owner=owner,
                namespace=namespace
            )
        )
        return "Saved new memory"

    return Tool(
        func=_register,
        name="register_user_preference_memory",
        description=(
            "Register a new memory or preference about the user, "
            "so it can be useful later as context.\n"
            "Args:\n"
            "    content: The information to save"
        )
    )

def build_memory_search_tool(ltm:LongTermMemory, owner:str, namespace:str):
    """
    Create a tool for agents to search existing memories.

    This factory function creates a tool that allows AI agents to retrieve
    relevant memories from the long-term memory system based on semantic
    similarity to a search query.

    Args:
        ltm (LongTermMemory): The memory system instance to use
        owner (str): User identifier for memory ownership
        namespace (str): Namespace to search within

    Returns:
        Tool: A configured tool for memory search
    """
    def _search(content:str):
        result = ltm.search(
            query_text=content,
            owner=owner,
            namespace=namespace,
            limit=3,
        )
        return str(tuple(zip(result.fragments, result.metadata['distances'])))

    return Tool(
        func=_search,
        name="search_user_preference_memory",
        description=(
            "Search for a stored memory or preference about the user, "
            "so it's useful as a context.\n"
            "Args:\n"
            "    content: The information to look for"
        )
    )

In [47]:
user_id = "952034f3-f57b-4346-b5d4-672b53851712"

udaplay.tools.append(build_memory_registration_tool(long_term_memory, user_id, "conversation"))
udaplay.tools.append(build_memory_search_tool(long_term_memory, user_id, "conversation"))

udaplay.instructions += """
## Long-Term Memory
- Whenever the user shares a preference, personal information, or something important, use the `register_memory` tool to save it.
- Before answering questions about user preferences or history, use `search_memory` to retrieve relevant information.
- Use memory to personalize your responses.
"""

run = udaplay.invoke("my favorite game is God of War", session_id=session_id)
state = run.get_final_state()

print("\n")
print("-" * 80)
print(f"Response: {state['messages'][-1].content}")
print("-" * 80)





[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


--------------------------------------------------------------------------------
Response: Nice choice! I’ve saved that memory: “Favorite game is God of War (series).”

Would you like me to tailor recommendations or info around God of War? Here are a few options:
- Quick overview: main games in the series, release years, and platforms
- Deep dive: differences between God of War (2005), God of War (2018), and God of War: Ragnarök
- Gameplay and lore: key mechanics, story beats, and character arcs
- Similar recommendations: other myth-inspired action-adventure games
- Latest news: any new releases, ports, or expansions in the GoW universe

Tell me which you prefer, or ask for something specific.
-------------------------------------

In [48]:
run = udaplay.invoke("I love super mario as well", session_id=session_id)
state = run.get_final_state()

print("\n")
print("-" * 80)
print(f"Response: {state['messages'][-1].content}")
print("-" * 80)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


--------------------------------------------------------------------------------
Response: Awesome—the GoW and Super Mario favorites are in your profile now.

Want some curated ideas around Mario? Here are a few good starting points across eras and styles:

- Modern 3D adventure (open-world feel, big playgrounds): Super Mario Odyssey (Nintendo Switch)
- Classic 2D platforming nostalgia: Super Mario World (SNES) or Super Mario Bros. 3 (NES)
- Milestones in 3D platforming: Super Mario 64 (N64) and Super Mario Galaxy (Wii)
- Story-rich RPG-ish entries: Paper Mario: The Thousand-Year Door (GameCube) or Super Paper Mario
- Fun, accessible co-op or

In [49]:
run = udaplay.invoke("What is my favorite game?", session_id=session_id)
state = run.get_final_state()

print("\n")
print("-" * 80)
print(f"Response: {state['messages'][-1].content}")
print("-" * 80)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


--------------------------------------------------------------------------------
Response: Your saved favorites are God of War (series) and Super Mario (franchise). 

If you want me to pick a primary favorite for focused recommendations, tell me which one should be the main—or I can keep tailoring suggestions that mix both depending on what you’re in the mood for.
--------------------------------------------------------------------------------
